# KOL Ascension Log Wrapper (v2.1)
_A Styling Project by Aaron M. (AKA Captain Scotch, #437479)_

Hey all! This is a wrapper meant to isolate and analyze Ascension logs via Python, formatting everything into a clean Excel wrapper. Originally, this was simply an overlay of [CKB's run log parser in ASH](https://kolmafia.us/showthread.php?22963-RunLogSummary), which is fantastic, but as I'm more of a Python dev this quickly enlarged into a broader project that turned into a personal goal to build a Python log parser. That's in development right now! I tried to make it as clean as possible; let me know if you have any questions.

In [1]:
import pandas as pd
import requests

In [2]:
# Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
kolmafDir = '/Users/amcguire/Library/Application Support/KoLmafia/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
kolName   = 'Captain Scotch'
runDates  = ['20190416','20190417'] 
runNotes  = 'blah blah blah blah /n blah blah blah BLAH /n blah blah, blah blah, Baaaaaah'

In [3]:
# Read in the last parsed run via RunLogSum.ash, for testing purposes
oldRun = pd.read_csv('{}data/{}-runlog_0112.txt'.format(kolmafDir,kolName),sep='\t')

In [4]:
# These are, all things considered, extremely small files. So I just read the whole
#   thing into memory so that it's all accessible without finicky itertools stuff.

# Populate our session dictionary
dDict = {}

for day, session in enumerate(runDates):
    # Python is zero-indexed, so we increment day by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    dDict[day+1] = open('{}sessions/{}_{}.txt'.format(kolmafDir, kolName.replace(" ","_"), session)).read()
    
    # Remove everything before Valhalla & after freeing Ralph
    if day == 0:
        start = dDict[1].find('Welcome to Valhalla!')
        dDict[1] = dDict[1][start:]
        
    else:
        end   = dDict[day+1].find('Freeing King Ralph')
        dDict[day+1] = dDict[day+1][:end]

# Also, to minimize server hits, grab a few files from the Mafia SVN data
mafCombats = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/combats.txt')
mafMonster = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/monsters.txt')
mafMods    = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/modifiers.txt')

In [5]:
mafEncounters = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/encounters.txt')

In [6]:
# This is from an effort a while ago to figure out how to build a script that knows where things "should" be.
yolo = {}
for x, line in enumerate(mafEncounters.text.split('\n')):
    try: 
        temp = line.split('\t')
        if len(temp) == 3:
            yolo[x] = {'LOCATION':temp[0], 'ADVTYPE':temp[1], 'NAME':temp[2]}
    except:
        print('error @ {}'.format(x))
    

In [7]:
findString = 'round 15'
buffer     = 3
day        = 1

# simple print w/ buffer for finding things within the logs
for ct, x in enumerate(dDict[day].split(sep='\n')):
    if findString.lower() in x.lower(): 
        print('===========\n'+
              '\n'.join(dDict[day].split(sep='\n')[(ct-buffer):(ct+buffer)])+
              '\n===========\n')

In [8]:
def monsterLocation(mon):
    ''' Sub-function that uses KOLMafia's combats directory to ascertain
        where a monster is likely to occur (used for establishing wanderer
        likelihood in the absence of full adventure parsing; was used for
        initial tests of digitize capture). Eventually will use this for
        full error-checking of the parser. '''
    
    locs = []
    
    mon = str(mon).strip()
    
    # Read in combats.txt and reformat it via split
    for line in mafCombats.text.split('\n'):
        if   mon in line: 
            locs = locs + [line.split('\t')[0]]
            
        # For some reason, Camel's Toe shows up as "The" Camel's Toe when
        #   you genie wish; this checks for and removes extraneous "The"
        elif mon.lower().replace('the ','') in line.lower():
                locs = locs + [line.split('\t')[0]]
    
    if locs == []: locs = ['FOOTAGE NOT FOUND']
        
    return locs
    
monsterLocation('Green Ops Soldier')

['The Battlefield (Frat Uniform)']

In [9]:
def cleanNames(monName):
    ''' This won't be much of an issue going forward, but
        the Intergnat had an annoying habit of modifying 
        monster names. This attempts to clean that particular 
        detritus to extract pure names. I'm assuming this
        will come back in other forms as well.
        
        Features to add: 
        
            - parse OCRS modifiers 
            - parse "Yes, Can Has" effect'''
    
    # All the strings added to monster names. Haven't added 
    #   OCRS modifiers yet, but intend to. Also would like 
    #   to include rewriting the can has skill, in case it
    #   comes up through the crazy horse, but haven't yet
    
    gnatList = ['NAMED NEIL', 'ELDRITCH HORROR ', 'AND TESLA',
                'WITH BACON!!!', 'WITH SCIENCE']
    
    ocrsMods = []
    
    replList = gnatList + ocrsMods
    
    for repl in replList:
        monName = monName.replace(repl,'')
        
    # Insert renaming for that one cheeseburger skill here
    
    return monName.strip()

In [10]:
def extractWanderers(sessionLogs):
    ''' Quick yet annoying function to attempt to ascertain wanderers. The 
        Digitize parser (unfortunately) doesn't work particularly well yet;
        when ascertaining digitize usage I usually just look over the print &
        mental math for the spreadsheet. The enamorang parser works better,
        which is good, since it's currently the only standard "choice" 
        wanderer as of 1/19. 
        
        Features to add: 
        
            - include detection of vote wanderers 
            - once the XML turn parser is done, revise this to reference it 
            - (you basically fuckin did that already in this hacky POS) 
            - doesn't properly handle LOV over two days '''
    
    wandererDict = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        wandererDict[day] = {}
        
        # Tracking digitized monsters is a pain in the ass. Luckily, simply
        #   extracting who they are can be pretty easy with some rough 
        #   nested loops.
        
        digiMons = []
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'casts DIGITIZE!' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                digiMons = digiMons + [monName]
    
        for mon in digiMons:
            
            # Now that we have that list, we want to figure out how many times
            #   they were fought. To do this, we'll need to compare the zone
            #   they originated from to the zones they have been found in.
            #   This method is meh, but kind of works, in the absence of Mafia 
            #   saving intro messages for digitized monsters. 
            
            monLoc = monsterLocation(mon)
            
            # So, cool unintentional thing here; when Mafia logs Witchess, it
            #   totally messes up the turn/combat statement, because it treats
            #   it differently. This is great! It means digitized Witchess
            #   pieces are super easy to find.
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        print('  Could not parse {}'.format(data))
    
        monLOVs = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'uses the LOV Enamorang' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                        
                        try: 
                            turn = int(dataSplit[0][1:])
                            loct = dataSplit[1]
                        except:
                            turn = -999
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                monLOVs[turn] = monName
                
        for turn, mon in monLOVs.items():
            
            # Similar process for above, with a few small items. Here, I am
            #   checking turn differential to make positively sure it is the
            #   enamorang'd monster; it has to be 14 turns! 
            
            monLoc = monsterLocation(mon)
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        foundTurn = int(data[0][1:])
                        currLoct = data[1]
                        # For testing
                        #print('Turn {}: fought {} @ {}.'.format(foundTurn,mon,loct))
                    except:
                        # Warn that something was unparsable 
                        print('  Could not parse {}'.format(data))
                    
                    if turn != foundTurn:
                        wandererDict[day][mon] = {'Type':'Enamorang',
                                         'Location':currLoct,
                                         'Turn':foundTurn}
    
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wandererDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wandererDict.keys()],sort=False).T
    
extractWanderers(dDict)

Day #1                     Day #2  \
                             Type   Location Turn       Type   
ninja snowman assassin  Enamorang  The Oasis  192        NaN   
Green Ops Soldier             NaN        NaN  NaN  Enamorang   

                                                             
                                              Location Turn  
ninja snowman assassin                             NaN  NaN  
Green Ops Soldier       The Battlefield (Frat Uniform)  394

In [11]:
def extractFreeKills(sessionLogs):
    ''' Quick function to ascertain freekills '''
    
    

In [12]:
def extractBanishesAndSniffs(sessionLogs, out='banishes'):
    ''' Quick function to ascertain banishes, sniffs, & macrometeorites '''
    
    banishList = ['ASDON MARTIN: SPRING-LOADED FRONT BUMPER!', 
                  'REFLEX HAMMER!', 
                  'KGB TRANQUILIZER DART!',
                  'THROW LATTE ON OPPONENT!',
                  'BALEFUL HOWL!']
    
    sniffList = ['OFFER LATTE TO OPPONENT!',
                 'PERCEIVE SOUL!']
    
    checkList = banishList + sniffList + ['MACROMETEORITE!']
    
    banishDict = {}
    sniffDict  = {}
    macroDict  = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        banishDict[day] = {}
        sniffDict[day]  = {}
        macroDict[day]  = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            for cond in checkList:
                if cond in row:
                    
                    if cond == 'MACROMETEORITE!':
                        nextMon = sessionLogs[day].split('\n')[ct+1].split('becomes')[1]
                    
                    x = 0

                    # Walk backwards through the file w/ a while loop 
                    while x < ct:
                        x +=1
                        monName = sessionLogs[day].split('\n')[ct-x]
                        if 'Encounter:' in monName:
                            dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                            try: 
                                turn = int(dataSplit[0][1:])
                                loct = dataSplit[1]
                            except:
                                turn = -999
                            break

                    monName = cleanNames(monName.replace('Encounter: ',''))
                    
                    if cond in banishList:
                        try:    num = max(list(banishDict[day].keys()))+1
                        except: num = 1
                        banishDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn}
                        
                    elif cond in sniffList:
                        try:    num = max(list(sniffDict[day].keys()))+1
                        except: num = 1
                        sniffDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn}
                        
                    else:
                        try:    num = max(list(macroDict[day].keys()))+1
                        except: num = 1
                        macroDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn,'NextMon':nextMon}
        
    if out=="sniffs":
        outDict = sniffDict
    elif out=="macros":
        outDict = macroDict
    else:
        outDict = banishDict
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in outDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in outDict.keys()],sort=False).T


In [13]:
extractBanishesAndSniffs(dDict,'banishes')

Day #1                                             \
                         Monster                                       Type   
1                        bookbat  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
2   animated mahogany nightstand                   THROW LATTE ON OPPONENT!   
3        Wardr&ouml;b nightstand                              BALEFUL HOWL!   
4             sabre-toothed goat                   THROW LATTE ON OPPONENT!   
5                     drunk goat                              BALEFUL HOWL!   
6                          Keese  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
7                         Goomba                              BALEFUL HOWL!   
8                            Zol                             REFLEX HAMMER!   
9                    Bullet Bill                     KGB TRANQUILIZER DART!   
10                   Protagonist  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
11        MagiMechTech MechaMech  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
12                      mad wino                     KGB TRANQUILIZER DART!   
13                      mad wino                             REFLEX HAMMER!   
14                   plaid ghost                             REFLEX HAMMER!   
15              gluttonous ghuol                   THROW LATTE ON OPPONENT!   
16                 Ninja Snowman  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
17                 Ninja Snowman                              BALEFUL HOWL!   
18                 Ninja Snowman                              BALEFUL HOWL!   
19    Ninja Snowman Weaponmaster                              BALEFUL HOWL!   
20         Ninja Snowman Janitor                              BALEFUL HOWL!   
21    Ninja Snowman Weaponmaster                              BALEFUL HOWL!   
22         Ninja Snowman Janitor                              BALEFUL HOWL!   
23         Procrastination Giant                              BALEFUL HOWL!   
24             Renaissance Giant                     KGB TRANQUILIZER DART!   
25                      cactuary                   THROW LATTE ON OPPONENT!   
26                           NaN                                        NaN   
27                           NaN                                        NaN   
28                           NaN                                        NaN   
29                           NaN                                        NaN   

                                                            \
                                             Location Turn   
1                                 The Haunted Library   27   
2                                 The Haunted Bedroom   37   
3                                 The Haunted Bedroom   37   
4                                         The Goatlet   50   
5                                         The Goatlet   57   
6                                         8-Bit Realm   70   
7                                         8-Bit Realm   70   
8                                         8-Bit Realm   70   
9                                         8-Bit Realm   70   
10                    The Penultimate Fantasy Airship  106   
11                    The Penultimate Fantasy Airship  136   
12                            The Haunted Wine Cellar  158   
13                            The Haunted Wine Cellar  158   
14                           The Haunted Laundry Room  159   
15                                 The Defiled Cranny  160   
16                          Lair of the Ninja Snowmen  171   
17                          Lair of the Ninja Snowmen  171   
18                          Lair of the Ninja Snowmen  171   
19                          Lair of the Ninja Snowmen  171   
20                          Lair of the Ninja Snowmen  171   
21                          Lair of the Ninja Snowmen  171   
22                          Lair of the Ninja Snowmen  171   
23  The Castle in the Clouds in the Sky (Ground Fl...  181   
24  The Castle in the Clou

In [14]:
extractBanishesAndSniffs(dDict,'sniffs')

Day #1                            \
                        Monster                      Type   
1    animated ornate nightstand  OFFER LATTE TO OPPONENT!   
2                    dairy goat            PERCEIVE SOUL!   
3                  Koopa Troopa            PERCEIVE SOUL!   
4                  Koopa Troopa  OFFER LATTE TO OPPONENT!   
5                dirty old lihc            PERCEIVE SOUL!   
6                Burly Sidekick            PERCEIVE SOUL!   
7                  Quiet Healer            PERCEIVE SOUL!   
8                  Quiet Healer            PERCEIVE SOUL!   
9   the cabinet of Dr. Limpieza            PERCEIVE SOUL!   
10            plaque of locusts  OFFER LATTE TO OPPONENT!   
11            plaque of locusts            PERCEIVE SOUL!   

                                                                 Day #2  \
                           Location Turn                        Monster   
1               The Haunted Bedroom   36                 dirty old lihc   
2                       The Goatlet   49            Blue Oyster cultist   
3                       8-Bit Realm   70            Blue Oyster cultist   
4                       8-Bit Realm   70                     red butler   
5                 The Defiled Niche  101                     red butler   
6   The Penultimate Fantasy Airship  107                    Bob Racecar   
7   The Penultimate Fantasy Airship  133              Green Ops Soldier   
8   The Penultimate Fantasy Airship  141              Green Ops Soldier   
9          The Haunted Laundry Room  159  spider (duck?) topiary animal   
10       The Arid, Extra-Dry Desert  186                            NaN   
11       The Arid, Extra-Dry Desert  186                            NaN   

                                                                   
                        Type                        Location Turn  
1             PERCEIVE SOUL!               The Defiled Niche  200  
2             PERCEIVE SOUL!    A Mob of Zeppelin Protesters  312  
3   OFFER LATTE TO OPPONENT!    A Mob of Zeppelin Protesters  312  
4   OFFER LATTE TO OPPONENT!                The Red Zeppelin  319  
5             PERCEIVE SOUL!                The Red Zeppelin  319  
6   OFFER LATTE TO OPPONENT!            Inside the Palindome  323  
7             PERCEIVE SOUL!          genie summoned monster  345  
8   OFFER LATTE TO OPPONENT!  The Battlefield (Frat Uniform)  381  
9             PERCEIVE SOUL!                       Twin Peak  430  
10                       NaN                             NaN  NaN  
11                       NaN                             NaN  NaN

In [15]:
extractBanishesAndSniffs(dDict,'macros')

Day #1                                               \
                        Monster             Type                    Location   
1                        Goomba  MACROMETEORITE!                 8-Bit Realm   
2                  Koopa Troopa  MACROMETEORITE!                 8-Bit Realm   
3                        Goomba  MACROMETEORITE!                 8-Bit Realm   
4                       Octorok  MACROMETEORITE!                 8-Bit Realm   
5                  Koopa Troopa  MACROMETEORITE!                 8-Bit Realm   
6                       Octorok  MACROMETEORITE!                 8-Bit Realm   
7                      mad wino  MACROMETEORITE!     The Haunted Wine Cellar   
8            swarm of fire ants  MACROMETEORITE!  The Arid, Extra-Dry Desert   
9   giant giant giant centipede  MACROMETEORITE!  The Arid, Extra-Dry Desert   
10                rock scorpion  MACROMETEORITE!  The Arid, Extra-Dry Desert   

                                            Day #2                   \
   Turn                 NextMon            Monster             Type   
1    70              a Tektite!   pygmy blowgunner  MACROMETEORITE!   
2    70              a Blooper!           boaraffe  MACROMETEORITE!   
3    73              a Blooper!      annoyed snake  MACROMETEORITE!   
4    74              a Blooper!      annoyed snake  MACROMETEORITE!   
5    75              a Blooper!  batwinged gremlin  MACROMETEORITE!   
6    79              a Blooper!  vegetable gremlin  MACROMETEORITE!   
7   158   a skeletal sommelier!      annoyed snake  MACROMETEORITE!   
8   186    a plaque of locusts!     spider gremlin  MACROMETEORITE!   
9   187    a plaque of locusts!      annoyed snake  MACROMETEORITE!   
10  188    a plaque of locusts!      annoyed snake  MACROMETEORITE!   

                                                           \
                                            Location Turn   
1                                    The Hidden Park  225   
2                                    The Hidden Park  231   
3                                       Sonofa Beach  233   
4                                       Sonofa Beach  244   
5   Next to that Barrel with Something Burning in it  251   
6   Next to that Barrel with Something Burning in it  252   
7                                       Sonofa Beach  255   
8                     Near an Abandoned Refrigerator  256   
9                                       Sonofa Beach  266   
10                                      Sonofa Beach  277   

                           
                  NextMon  
1             a boaraffe!  
2     a pygmy blowgunner!  
3       a lobsterfrogman!  
4       a lobsterfrogman!  
5    a vegetable gremlin!  
6    a batwinged gremlin!  
7       a lobsterfrogman!  
8    a batwinged gremlin!  
9       a lobsterfrogman!  
10      a lobsterfrogman!

In [16]:
def extractWishes(sessionLogs):
    ''' Quick function to ascertain wish usage '''
    
    wishDict = {}
    
    for day in sessionLogs.keys():
        
        wishDict[day] = {}
        
        # Wishes aren't quite as easy as pulls, which are easily snagged
        #   on one line with newline split logic, but they're certainly 
        #   easier than digitizations.
        
        # Combat parsing, utilizing the combat to pull monsters out
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "genie summoned monster" in row:
                turn = row[(row.find('[')+1):row.find(']')]
                wishMon = sessionLogs[day].split('\n')[ct+1].replace('Encounter: ','')
                wishMon = cleanNames(wishMon)
                wishDict[day][ct] = {'Type':'Fight',
                                     'Details':wishMon,
                                     'Turn':int(turn)}
        
        # Extra parsing, using the actual wish URL to fill others
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "&wish=" in row:
                wishString = row[(row.find('&wish=')+6):]
                try: 
                    
                    # Note we have to use ct+2 due to session logs
                    #   taking 2 lines to get from URL to fight. If
                    #   this ever changes, will need to change.
                    
                    wishDict[day][ct+2]['Type']
                    
                except:
                    
                    wishDict[day][ct] = {'Type':'Non-Fight',
                                         'Details':wishString,
                                         'Turn':'?'}

        # Rename the wishes from line # to wish #
        for i, val in enumerate(wishDict[day]):
            wishDict[day][i+1] = wishDict[day].pop(val)
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wishDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wishDict.keys()]).T

extractWishes(dDict)

Day #1                             Day #2  \
    Type            Details Turn       Type   
1  Fight  Sorority Operator   81      Fight   
2  Fight       mountain man  151  Non-Fight   
3  Fight     Baa'baa'bu'ran  195  Non-Fight   

                                                
                                  Details Turn  
1                       Green Ops Soldier  345  
2                        to+be+Dirty+Pear    ?  
3  to+be+Fifty+Ways+to+Bereave+Your+Lover    ?

In [17]:
def extractPulls(sessionLogs):
    ''' Quick function to extract normal run pulls into a table. '''
    
    pullDict = {}
    
    for day in sessionLogs.keys():
        
        # Find all your pulls; relies on sessions storing them as 'pull: '
        pulls = [i[6:] for i in sessionLogs[day].split('\n') if i.startswith('pull: ')]
        
        # Split out pulls into a 20 item list, for formatting
        numbs = [int(i[0:2].strip()) for i in pulls]
        pullDict[day] = []
        for count, pull in enumerate(pulls):
            pullDict[day] = pullDict[day] + [pull[1:].strip()]*numbs[count]
        
        if len(pullDict[day]) < 20:
            pullDict[day] = pullDict[day] + ['']*(20-len(pullDict[day]))
    
    # Renaming for simplicity/ease of pasting; 
    pullsOut = pd.DataFrame(pullDict).rename(
        index  = lambda x: '#{}'.format(x+1),
        columns= lambda x: 'Day #{}'.format(x))
    
    return pullsOut

extractPulls(dDict)

,Day #1,Day #2
#1,,
#2,,
#3,,
#4,,
#5,,
#6,,
#7,,
#8,,
#9,,
#10,,


In [18]:
def goblinParser(sessionLogs):
    ''' Figure out where/when the user encountered sausage goblins.
        This SHOULD NOT be used as a spading tool, as it does not
        properly track important things like how much meat was in
        the grinder and how many turns the grinder was used. It's
        just for me to think about where I dropped goblins in-run '''
    
    goblinDict = {}
    gobNum = 0
    
    for day in sessionLogs.keys():
        
        # I am keeping this a list in case we eventually find a 
        #   sausage goblin boss...
        
        for mon in ['sausage goblin']:
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    gobNum = gobNum + 1
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        # print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        # Gotta show that error!
                        print('  Could not parse {}'.format(data))
                    
                    goblinDict[gobNum] =      {'Turn': turn, 
                                               'Location': loct,
                                               'Day': day}
    
        
    return pd.DataFrame.from_dict(goblinDict).T
                        
goblinParser(dDict)

,Day,Location,Turn
1,1,The Spooky Forest,2
2,1,The Spooky Forest,12
3,1,The Outskirts of Cobb's Knob,24
4,1,The Haunted Bathroom,37
5,1,The Haunted Bathroom,53
6,1,The Haunted Bathroom,67
7,1,The Haunted Ballroom,97
8,1,The Neverending Party,104
9,1,The Black Forest,116
10,1,The Penultimate Fantasy Airship,131


## Unfinished functions, to be added over time

In [19]:
def buffSelection(sessionLogs):
    ''' Parse out the IOTM buffs selected in the selected run, by day/turn '''

In [20]:
def itemSelection(sessionLogs):
    ''' Parse out the IOTM items selected in the selected run, by day '''

In [21]:
def horseBuffSpading(sessionLogs):
    ''' Find out what buffs you get from the crazy horse, for a simulation
        project I'm working on to establish bounds on crazy horse value '''
